In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


# Import Mask RCNN
from mrcnn import visualize
import mrcnn.model as modellib

from samples.brixilated_lego import lego

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.__version__

2021-10-24 04:06:01.873978: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


'2.5.0'

In [2]:
ROOT_DIR = './'
os.chdir(ROOT_DIR)

In [3]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [4]:
class InferenceConfig(lego.LegoConfig().__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

In [5]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
LEGO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "snapshots", "weights","mask_rcnn_lego_0111.h5") # Comment out to use snapshot from latest training

model = modellib.MaskRCNN(mode="inference", model_dir="", config=config)
print("Loading weights ", LEGO_WEIGHTS_PATH)
model.load_weights(LEGO_WEIGHTS_PATH, by_name=True)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Instructions for updating:
Use fn_output_signature instead


2021-10-24 04:06:07.463072: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-24 04:06:07.490738: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-24 04:06:07.490778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: TRLPanda
2021-10-24 04:06:07.490787: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: TRLPanda
2021-10-24 04:06:07.490939: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.91.3
2021-10-24 04:06:07.490970: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2021-10-24 04:06:07.490979: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.91.3


Loading weights  ./snapshots/weights/mask_rcnn_lego_0111.h5


2021-10-24 04:06:08.566587: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-24 04:06:09.041378: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3492955000 Hz


In [6]:
IMG_DIR = os.path.join(ROOT_DIR, 'datasets', 'real_images')
img_gen = tf.keras.preprocessing.image.ImageDataGenerator()
generator = img_gen.flow_from_directory(IMG_DIR, target_size=(3297, 1908), class_mode=None, batch_size=1, shuffle=False)

num_images = len(os.listdir(os.path.join(IMG_DIR, 'imgs')))
print(num_images)

Found 5 images belonging to 1 classes.
5


In [21]:
# get dataset class names
DATASET = "val"
LEGO_DIR = os.path.join(ROOT_DIR, "datasets", "output", "coco_data")

dataset = lego.LegoDataset()
dataset.load_coco(LEGO_DIR, DATASET)
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

loading annotations into memory...
Done (t=6.49s)
creating index...
index created!
Images: 3029
Classes: ['BG', '2431', '3003', '3005', '3010', '3020', '3021', '3022', '3023', '3024', '3069', '3070', '3176', '3622', '3700', '3710', '3958', '4150', '4274', '6141', '11211', '11476', '11477', '15068', '15573', '22885', '24201', '24246', '25269', '29119', '29120', '33909', '35480', '36840', '47458', '47905', '85984', '87079', '87087', '87580', '93273', '98138', '99206']


In [ ]:
# run through images from generator
for img_idx, batch in enumerate(generator):
    
    if img_idx >= num_images:
        break
        
    results = model.detect(batch, verbose=0)
    r = results[0]

    ax = get_ax(1)
    visualize.display_instances(batch[0], r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax, title="Predictions")
    plt.axis('image')
    plt.savefig(f'output_{img_idx}.png', bbox_inches='tight') # save output
    plt.show()

In [8]:
img_url = 'https://thumbs.dreamstime.com/b/lego-blocks-17854970.jpg'
image_path = 'lego-blocks-17854970.jpg'
image = tf.keras.preprocessing.image.load_img(image_path)
image_data = tf.keras.preprocessing.image.img_to_array(image)
results = model.detect([image_data], verbose=0)[0]
results

/home/panda/.conda/envs/lego/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


{'rois': array([[273, 369, 303, 397]], dtype=int32),
 'class_ids': array([32], dtype=int32),
 'scores': array([0.7590335], dtype=float32),
 'masks': array([[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        ...,
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]]])}

In [17]:
response = {}
response['class_ids'] = results['class_ids'].tolist()
response['rois'] = results['rois'].tolist()
response['scores'] = results['scores'].tolist()
response

{'class_ids': [32],
 'rois': [[273, 369, 303, 397]],
 'scores': [0.7590335011482239]}

In [28]:
import json
output = []
for _id, _name in enumerate(dataset.class_names):
    output.append({"id": _id, "name": _name})
print(json.dumps(output, indent=2, sort_keys=True))

[
  {
    "id": 0,
    "name": "BG"
  },
  {
    "id": 1,
    "name": "2431"
  },
  {
    "id": 2,
    "name": "3003"
  },
  {
    "id": 3,
    "name": "3005"
  },
  {
    "id": 4,
    "name": "3010"
  },
  {
    "id": 5,
    "name": "3020"
  },
  {
    "id": 6,
    "name": "3021"
  },
  {
    "id": 7,
    "name": "3022"
  },
  {
    "id": 8,
    "name": "3023"
  },
  {
    "id": 9,
    "name": "3024"
  },
  {
    "id": 10,
    "name": "3069"
  },
  {
    "id": 11,
    "name": "3070"
  },
  {
    "id": 12,
    "name": "3176"
  },
  {
    "id": 13,
    "name": "3622"
  },
  {
    "id": 14,
    "name": "3700"
  },
  {
    "id": 15,
    "name": "3710"
  },
  {
    "id": 16,
    "name": "3958"
  },
  {
    "id": 17,
    "name": "4150"
  },
  {
    "id": 18,
    "name": "4274"
  },
  {
    "id": 19,
    "name": "6141"
  },
  {
    "id": 20,
    "name": "11211"
  },
  {
    "id": 21,
    "name": "11476"
  },
  {
    "id": 22,
    "name": "11477"
  },
  {
    "id": 23,
    "name": "15068"
  }

In [11]:
results

{'rois': array([[273, 369, 303, 397]], dtype=int32),
 'class_ids': array([32], dtype=int32),
 'scores': array([0.7590335], dtype=float32),
 'masks': array([[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        ...,
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]]])}

In [10]:
results['class_ids']

array([32], dtype=int32)